# Flow Cytometry Analysis code - Template


## Set up paths for output

Before we begin processing this data, I want to make sure I have an output folder where I can save the raw data, as well as any figures that we generate.

In [1]:
def make_paths(dates):
    import os
    
    paths = {}
    paths['IO'] = os.path.relpath(str(dates[0])) #A folder with the date name is made during data unpacking
    paths['Figures'] = os.path.join(paths['IO'],'Figures')
    paths['Beads'] = os.path.join(paths['Figures'],'Beads')
    paths['Samples'] = os.path.join(paths['Figures'],'Samples')
    paths['WCS'] = os.path.join(paths['Figures'],'WCS')
    paths['No_WCS'] = os.path.join(paths['Figures'],'No_WCS')
    paths['Histograms'] = os.path.join(paths['Figures'],'Histograms')
    paths['Heatmaps'] =  os.path.join(paths['Figures'],'Heatmaps')
    
    #The following block makes a folder for the figures
    #It also handles a common error that can happen if the folder already exists.
    for path in list(paths.values()):
        if not os.path.exists(path):
            os.mkdir(path)
            print(path+' created')
        else:
            print(path+' already exists')
        
    return paths

## Generate beads transfer functions for each replicate

In [2]:
def flow_analyze(dates, paths):
    import FlowCal
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    
    input_files = {}
    for date in dates:
        input_files[date] = '{}/{}_key.xlsx'.format(date, date)
        
    fc_output = pd.DataFrame()
    for date in input_files.keys():
        input_file = pd.read_excel(input_files[date],'Samples')
        input_file['Date'] = date
        fc_output = fc_output.append(input_file)
    
    fc_output['Gated MEFL'] = None
    fc_output['Ungated MEFL'] = None
    fc_output['Gated MECY'] = None
    fc_output['Ungated MECY'] = None

    beads_functions = {}
    
    for date in input_files.keys():
        print('Generating beads standard curve for '+date+'...',end='')
        beads = pd.read_excel(input_files[date],'Beads')
        beads = beads.set_index('ID')
        
        for beads_id in beads.index:
            # Adapted from FlowCal documentation
            # https://taborlab.github.io/FlowCal/python_tutorial/mef.html
            mef_fl1 = [(np.nan if x=='None' else int(x)) for x in beads.loc[beads_id, 'FL1 MEF Values'].split(', ')]
            #mef_fl2 = [(np.nan if x=='None' else int(x)) for x in beads.loc[beads_id, 'FL3 MEF Values'].split(', ')]
            
            b = FlowCal.io.FCSData('/'.join(input_files[date].split('/')[:-1])
                                   +'/'+beads.loc[beads_id, 'File Path'])
            b = FlowCal.transform.to_rfi(b)
            
            # Density gate with gate fraction 0.3
            #print(b)
            b_g, __, c = FlowCal.gate.density2d(b,
                                                channels=['FSC-H', 'SSC-H'],
                                                gate_fraction=0.3,
                                                full_output=True)
            
            to_mef = FlowCal.mef.get_transform_fxn(b_g,
                                                   mef_values=[mef_fl1],
                                                   mef_channels=['FL1-H'],
                                                   plot=True, plot_dir=paths['Beads'])
            #If collecting mCherry measurements, 
            #change to mef_values=[mef_fl1,mef_fl2] and mef_channels=['FL1-H','FL2-H']
            #and uncomment the related code above and below
            
            beads_functions[(date, beads_id)] = to_mef
            
            #FL1 Beads plots
            FlowCal.plot.density_and_hist(b, gated_data=b_g, gate_contour=c, density_channels=['FSC-H','SSC-H'],
                                          density_params={'mode':'scatter'},hist_channels=['FL1-H'],
                                          savefig=paths['Beads']+'/Beads {}.png'.format(beads_id))
            #FL2 Beads plots
            #FlowCal.plot.density_and_hist(b, gated_data=b_g, gate_contour=c, density_channels=['FSC-H','SSC-H'],
            #                              density_params={'mode':'scatter'},hist_channels=['FL2-H'],
            #                              savefig=paths['Beads']+'/Beads_FL3 {}.png'.format(beads_id))
        print('Done!')
    sample_data = {}
    for date in input_files.keys():
        print('Analyzing samples for '+date+'...',end='')
        fc_date = fc_output[fc_output['Date']==date]
        
        for i in fc_date.index:
            # Adapted from FlowCal documentation:
            # https://taborlab.github.io/FlowCal/python_tutorial/mef.html
            
            s = FlowCal.io.FCSData('/'.join(input_files[date].split('/')[:-1])
                                   +'/'+fc_date.loc[i,'File Path'])
            
            s = FlowCal.transform.to_rfi(s)
            
            to_mef = beads_functions[(date, fc_date.loc[i, 'Beads ID'])]
            s = to_mef(s, channels=['FL1-H'])
            
            #Like before add back 'FL2-H' to channels if using mCherry
            ungated = s
            fc_output.at[i,'Ungated MEFL'] = ungated
            
            #s = FlowCal.gate.high_low(s,channels=['FSC-H'],high=5000)
            # Density gate with gate fraction 0.85
            s, mask, contour = FlowCal.gate.density2d(s, channels=['FSC-H', 'SSC-H'],
                                                      gate_fraction = 0.3,full_output=True)
            
            fc_output.at[i,'Gated MEFL'] = s
            
            plasmid = fc_output.at[i,'Plasmid(s)']
            condition = fc_output.at[i,'Ligand']
            FlowCal.plot.density_and_hist(ungated, gated_data=s, gate_contour=contour, density_channels=['FSC-H','SSC-H'],
                                          density_params={'mode':'scatter'},hist_channels=['FL1-H'],
                                          savefig=paths['Samples']+'/FL1_Sample '+str(i)+'_'+plasmid+'_'+condition.replace('/','_')+'.png')
            
            #FlowCal.plot.density_and_hist(ungated, gated_data=s, gate_contour=contour, density_channels=['FSC-H','SSC-H'],
            #                              density_params={'mode':'scatter'},hist_channels=['FL2-H'],
            #                              savefig=paths['Samples']+'/FL3_Sample '+str(i)+'_'+plasmid+'_'+condition.replace('/','_')+'.png')
            
            sample_data[(fc_date.loc[i,'File Path'], date)] = s
    
    fc_output['Gated FL1 Mean'] = fc_output.apply(lambda x:
                                                  sample_data[(x['File Path'], x['Date'])][:, 'FL1-H'].mean(), axis=1)
    #fc_output['Gated FL2 Mean'] = fc_output.apply(lambda x:
    #                                              sample_data[(x['ID'], x['Date'])][:, 'FL2-H'].mean(), axis=1)
    print('Done!')
    print('Saving analyzed output to csv in '+dates[0]+' folder')
    fc_output.to_csv(paths['IO']+'/'+str(dates[0])+'_output.csv')
    
    return fc_output

## Make Histograms for all samples
The following code should be generalizable for any experiment which follows the same excel key format.

In [1]:
def make_FL1_histograms(fc_output, paths):
    import FlowCal
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    
    """
    Plot FL1 Histograms with FlowCal. Handles experiments with aTc/IPTG, SK & RR promoters, neither, or both.

    Parameters
    ----------
    fc_output
        A pandas dataframe containing flow data including information from the experiment key. Can be obtained as
        the return from AnalyzeFlow(date).
    """
    print('Plotting histograms...')
    
    tcss=list(set(fc_output['Plasmid(s)']))
    if 'None' in tcss:
        tcss.remove('None')
        #We don't need to make a separate histogram for autofluorescence sample.
        
    ligands={}
    for tcs in tcss:
        ligands[tcs] = list(set(fc_output['Ligand']))
        if 'Auto' in ligands[tcs]:
            ligands[tcs].remove('Auto')
    
    ligand_concs = {}
    for tcs in tcss:
        for ligand in ligands[tcs]:
            ligand_concs[ligand] = list(set(fc_output.loc[fc_output['Ligand']==ligand]['[Ligand] (mM)'].dropna()))
            if 0 in ligand_concs[ligand]:
                ligand_concs[ligand].remove(0)
            
            #preallocate dicts to put dataframes in
    minus = {}
    plus = {}
    folds = {}
    minus_hists = {}
    plus_hists = {}
            
    
    if ('aTc (ng/mL)' in fc_output.columns) & ('IPTG (uM)' in fc_output.columns):
        #Assume we will always have aTc & IPTG co-occurring
        atc = {}
        iptg = {}
        
        for tcs in tcss:
            for ligand in ligands[tcs]:
                if fc_output.loc[(fc_output['Ligand']==ligand)&(fc_output['Plasmid(s)']==tcs)].empty:
                    #This block will prevent errors from the code looking for samples that don't exist
                    #With this structure it shouldn't be necessary but best not to risk it I think.
                    continue
                atc[tcs, ligand] = list(set(fc_output.loc[(fc_output['Plasmid(s)']==tcs)&(fc_output['Ligand']==ligand)]['aTc (ng/mL)'].dropna()))
                atc[tcs, ligand].sort()
                iptg[tcs, ligand] = list(set(fc_output.loc[(fc_output['Plasmid(s)']==tcs)&(fc_output['Ligand']==ligand)]['IPTG (uM)'].dropna()))
                iptg[tcs, ligand].sort(reverse=True)
                
                for conc in ligand_concs[ligand]:
                    minus[tcs, ligand, conc] = pd.DataFrame(data=np.empty([len(atc[tcs,ligand]),len(iptg[tcs,ligand])]),
                                                            index=atc[tcs,ligand],columns=iptg[tcs,ligand])
                    plus[tcs, ligand, conc] = pd.DataFrame(data=np.empty([len(atc[tcs,ligand]),len(iptg[tcs,ligand])]),
                                                           index=atc[tcs,ligand],columns=iptg[tcs,ligand])
                    folds[tcs, ligand, conc] = pd.DataFrame(data=np.empty([len(atc[tcs,ligand]),len(iptg[tcs,ligand])]),
                                                            index=atc[tcs,ligand],columns=iptg[tcs,ligand])
                    minus_hists[tcs, ligand, conc] = pd.DataFrame(data=np.empty([len(atc[tcs,ligand]),len(iptg[tcs,ligand])]),
                                                                  index=atc[tcs,ligand],columns=iptg[tcs,ligand])
                    plus_hists[tcs, ligand, conc] = pd.DataFrame(data=np.empty([len(atc[tcs,ligand]),len(iptg[tcs,ligand])]),
                                                                 index=atc[tcs,ligand],columns=iptg[tcs,ligand])
                
                    

                
                for a in atc[tcs,ligand]:
                    for i in iptg[tcs,ligand]:
                        #Need to clear all the columns of minus_hists and plus_hists before you can store anything in them
                        #minus_hists[tcs, ligand, conc][i] = [[]]
                        #plus_hists[tcs, ligand, conc][i] = [[]]
                        
                        for conc in ligand_concs[ligand]:
                            #print(tcs+' '+str(a)+' '+str(i)+' '+str(conc))
                            if fc_output.loc[(fc_output['Ligand']==ligand)&(fc_output['Plasmid(s)']==tcs)&
                                             (fc_output['aTc (ng/mL)']==a)&(fc_output['IPTG (uM)']==i)&
                                             (fc_output['[Ligand] (mM)']==conc)].empty:
                                #This block will prevent errors from the code looking for samples that don't exist
                                #With this structure it shouldn't be necessary but best not to risk it I think.
                                continue
                            
                            fig,ax = plt.subplots()
                            
                            if fc_output.loc[(fc_output['Ligand']==ligand)&(fc_output['Plasmid(s)']==tcs)&
                                             (fc_output['aTc (ng/mL)']==a)&(fc_output['IPTG (uM)']==i)&
                                             (fc_output['[Ligand] (mM)']==0)].empty:
                                hist_data = [fc_output.loc[(fc_output['Ligand']=='Water')&(fc_output['[Ligand] (mM)']==conc)&
                                                           (fc_output['Plasmid(s)']==tcs)&(fc_output['aTc (ng/mL)']==a)&
                                                           (fc_output['IPTG (uM)']==i)]['Gated MEFL'].values[0],
                                             fc_output.loc[(fc_output['Ligand']==ligand)&(fc_output['[Ligand] (mM)']==conc)&
                                                           (fc_output['Plasmid(s)']==tcs)&(fc_output['aTc (ng/mL)']==a)&
                                                           (fc_output['IPTG (uM)']==i)]['Gated MEFL'].values[0]]
                                
                                minus_mefl = fc_output.loc[(fc_output['Ligand']=='Water')&(fc_output['[Ligand] (mM)']==conc)&
                                                       (fc_output['Plasmid(s)']==tcs)&(fc_output['aTc (ng/mL)']==a)&
                                                       (fc_output['IPTG (uM)']==i)]['Gated FL1 Mean'].values[0]
                                plus_mefl = fc_output.loc[(fc_output['Ligand']==ligand)&(fc_output['[Ligand] (mM)']==conc)&
                                                          (fc_output['Plasmid(s)']==tcs)&(fc_output['aTc (ng/mL)']==a)&
                                                          (fc_output['IPTG (uM)']==i)]['Gated FL1 Mean'].values[0]
                            else:
                                hist_data = [fc_output.loc[(fc_output['Ligand']==ligand)&(fc_output['[Ligand] (mM)']==0)&
                                                           (fc_output['Plasmid(s)']==tcs)&(fc_output['aTc (ng/mL)']==a)&
                                                           (fc_output['IPTG (uM)']==i)]['Gated MEFL'].values[0],
                                             fc_output.loc[(fc_output['Ligand']==ligand)&(fc_output['[Ligand] (mM)']==conc)&
                                                           (fc_output['Plasmid(s)']==tcs)&(fc_output['aTc (ng/mL)']==a)&
                                                           (fc_output['IPTG (uM)']==i)]['Gated MEFL'].values[0]]
                                minus_mefl = fc_output.loc[(fc_output['Ligand']==ligand)&(fc_output['[Ligand] (mM)']==0)&
                                                           (fc_output['Plasmid(s)']==tcs)&(fc_output['aTc (ng/mL)']==a)&
                                                           (fc_output['IPTG (uM)']==i)]['Gated FL1 Mean'].values[0]
                                plus_mefl = fc_output.loc[(fc_output['Ligand']==ligand)&(fc_output['[Ligand] (mM)']==conc)&
                                                          (fc_output['Plasmid(s)']==tcs)&(fc_output['aTc (ng/mL)']==a)&
                                                          (fc_output['IPTG (uM)']==i)]['Gated FL1 Mean'].values[0]
                            
                            FlowCal.plot.hist1d(hist_data,channel='FL1-H',facecolor=['#00000088','#07af6988'],
                                                legend=True,legend_labels=['Water',str(conc)+' mM '+ligand],
                                                xlabel='sfGFP Fluorescence (MEFL)',
                                                title=tcs+' '+str(a)+' ng/mL aTc, '+str(i)+' $\mu$M IPTG'
                                                +'\n'+str(conc)+' mM '+ligand,
                                                savefig=paths['Figures']+'/Histograms/{}_hist.png'.format(tcs
                                                                                                          +'_'+str(conc)
                                                                                                          +'_'+str(ligand)
                                                                                                          +'_a'+str(a)
                                                                                                          +'_i'+str(i)))
                            
                            
                            fold_change = plus_mefl/minus_mefl
                            
                            minus[tcs,ligand,conc].at[a,i] = minus_mefl
                            plus[tcs,ligand,conc].at[a,i] = plus_mefl
                            folds[tcs,ligand,conc].at[a,i] = fold_change
                            
                            #minus_hists[tcs,ligand,conc].at[a,i] = hist_data[0]
                            #plus_hists[tcs,ligand,conc].at[a,i] = hist_data[1]
                            
    #Next look for samples with no aTc/IPTG but with SK/RR Promoters specified
    #Importantly, as written, I don't require them to be mutually exclusive
    if ('SK Promoter' in fc_output.columns) & ('RR Promoter' in fc_output.columns):
        #Assume that if an SK promoter is specified, an RR promoter will be as well
        sk_prom = {}
        rr_prom = {}
        
        for tcs in tcss:
            for ligand in ligands[tcs]:
                sk_prom[tcs, ligand] = list(set(fc_output.loc[(fc_output['Plasmid(s)']==tcs)&(fc_output['Ligand']==ligand)]['SK Promoter'].dropna()))
                rr_prom[tcs, ligand] = list(set(fc_output.loc[(fc_output['Plasmid(s)']==tcs)&(fc_output['Ligand']==ligand)]['RR Promoter'].dropna()))
                
                for conc in ligand_concs[ligand]:
                    minus[tcs, ligand,conc] = pd.DataFrame(data=np.empty([len(rr_prom[tcs,ligand]),len(sk_prom[tcs,ligand])]),
                                                      index=rr_prom[tcs,ligand],columns=sk_prom[tcs,ligand])
                    plus[tcs, ligand,conc] = pd.DataFrame(data=np.empty([len(rr_prom[tcs,ligand]),len(sk_prom[tcs,ligand])]),
                                                     index=rr_prom[tcs,ligand],columns=sk_prom[tcs,ligand])
                    folds[tcs, ligand,conc] = pd.DataFrame(data=np.empty([len(rr_prom[tcs,ligand]),len(sk_prom[tcs,ligand])]),
                                                      index=rr_prom[tcs,ligand],columns=sk_prom[tcs,ligand])
                    minus_hists[tcs, ligand,conc] = pd.DataFrame(data=np.empty([len(rr_prom[tcs,ligand]),len(sk_prom[tcs,ligand])]),
                                                                 index=rr_prom[tcs,ligand],columns=sk_prom[tcs,ligand])
                    plus_hists[tcs, ligand,conc] = pd.DataFrame(data=np.empty([len(rr_prom[tcs,ligand]),len(sk_prom[tcs,ligand])]),
                                                                index=rr_prom[tcs,ligand],columns=sk_prom[tcs,ligand])
        
        for tcs in tcss:
            for ligand in ligands[tcs]:
                for s in sk_prom[tcs,ligand]:
                    for r in rr_prom[tcs,ligand]:
                        if fc_output.loc[(fc_output['Ligand']==ligand)&(fc_output['Plasmid(s)']==tcs)&
                                         (fc_output['SK Promoter']==s)&(fc_output['RR Promoter']==r)].empty:
                            #If testing a mixed set of ligands + TCSs where not all TCSs encounter 
                            #all ligands/all [aTc]/all [IPTG],
                            #This block will prevent errors from the code looking for samples that don't exist
                            continue
                        
                        for conc in ligand_concs[ligand]:
                            fig,ax = plt.subplots()
                            hist_data = [fc_output.loc[(fc_output['Ligand']==ligand)&(fc_output['[Ligand] (mM)']==0)&
                                                       (fc_output['Plasmid(s)']==tcs)&(fc_output['SK Promoter']==s)&
                                                       (fc_output['RR Promoter']==r)]['Gated MEFL'].values[0],
                                         fc_output.loc[(fc_output['Ligand']==ligand)&(fc_output['[Ligand] (mM)']==conc)&
                                                       (fc_output['Plasmid(s)']==tcs)&(fc_output['SK Promoter']==s)&
                                                       (fc_output['RR Promoter']==r)]['Gated MEFL'].values[0]]
                            
                            FlowCal.plot.hist1d(hist_data,channel='FL1-H',facecolor=['#00000088','#07af6988'],
                                                legend=True,legend_labels=['Water',str(conc)+' mM '+ligand],
                                                xlabel='sfGFP Fluorescence (MEFL)',
                                                title=tcs+' '+str(conc)+' mM '+ligand,
                                                savefig=paths['Figures']+'/Histograms/{}_hist.png'.format(tcs
                                                                                                          +'_'+str(conc)
                                                                                                          +'_'+str(ligand)
                                                                                                          +'_s'+str(s)
                                                                                                          +'_r'+str(r)))
                            minus_mefl = fc_output.loc[(fc_output['Ligand']==ligand)&(fc_output['[Ligand] (mM)']==0)&
                                                       (fc_output['Plasmid(s)']==tcs)&(fc_output['RR Promoter']==r)&
                                                       (fc_output['SK Promoter']==s)]['Gated FL1 Mean'].values[0]
                            plus_mefl = fc_output.loc[(fc_output['Ligand']==ligand)&(fc_output['[Ligand] (mM)']==conc)&
                                                      (fc_output['Plasmid(s)']==tcs)&(fc_output['RR Promoter']==r)&
                                                      (fc_output['SK Promoter']==s)]['Gated FL1 Mean'].values[0]
                            fold_change = plus_mefl/minus_mefl
                            
                            minus[tcs,ligand,conc].at[r,s] = minus_mefl
                            plus[tcs,ligand,conc].at[r,s] = plus_mefl
                            folds[tcs,ligand,conc].at[r,s] = fold_change
                            minus_hists[tcs,ligand,conc].at[r,s] = hist_data[0] 
                            plus_hists[tcs,ligand,conc].at[r,s] = hist_data[1]
                            
                            
    #Finally, if neither aTc/IPTG nor SK/RR Promoters are specified, attempt to make histograms based 
    #solely on TCS & Ligand
    if ('aTc (ng/mL)' not in fc_output.columns)&('IPTG (uM)' not in fc_output.columns)&('SK Promoter' not in fc_output.columns)&('RR Promoter' not in fc_output.columns):
        for tcs in tcss:
            for ligand in ligands[tcs]:
                if fc_output.loc[(fc_output['Ligand']==ligand)&(fc_output['Plasmid(s)']==tcs)].empty:
                    #If testing a mixed set of ligands + TCSs where not all TCSs encounter 
                    #all ligands/all [aTc]/all [IPTG],
                    #This block will prevent errors from the code looking for samples that don't exist
                    continue
                
                for conc in ligand_concs[ligand]:
                    fig,ax = plt.subplots()
                    hist_data = [fc_output.loc[(fc_output['Ligand']==ligand)&(fc_output['[Ligand] (mM)']==0)&
                                               (fc_output['Plasmid(s)']==tcs)]['Gated MEFL'].values[0],
                                 fc_output.loc[(fc_output['Ligand']==ligand)&(fc_output['[Ligand] (mM)']==conc)&
                                               (fc_output['Plasmid(s)']==tcs)]['Gated MEFL'].values[0]]
                    
                    FlowCal.plot.hist1d(hist_data,channel='FL1-H',facecolor=['#00000088','#07af6988'],
                                        legend=True,legend_labels=['Water',str(conc)+' mM '+ligand],
                                        xlabel='sfGFP Fluorescence (MEFL)',
                                        title=tcs+' '+str(conc)+' mM '+ligand,
                                        savefig=paths['Figures']+'/Histograms/{}_hist.png'.format(tcs
                                                                                                  +'_'+str(conc)
                                                                                                  +'_'+str(ligand)))
    
    print('Histograms plotted and saved successfully')
    dates = list(set(fc_output['Date']))
    #absorbance_heatmap(dates, paths, tcss, ligand, ligand_concs[ligand])
    return minus, plus, folds, minus_hists, plus_hists
    

## Absorbance Heatmap

Make sure that your (date)_Abs.xlsx file has the proper format

In [ ]:
def heatmap(data, row_labels, col_labels,xlabel, ylabel,title,vmin,vmax,paths,ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (M, N).
    row_labels
        A list or array of length M with the labels for the rows.
    col_labels
        A list or array of length N with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """
    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, norm=SymLogNorm(linthresh=0.01, vmin=vmin,vmax=vmax), **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    #pcm = ax.pcolormesh(data, cmap=plt.cm.get_cmap('bwr_r'), 
    #norm=SymLogNorm(linthresh=0.01, vmin=1/10.,vmax=10., base=2.))
    #cbar = ax.figure.colorbar(pcm,ax=ax,**cbar_kw)
    #cbar.locator=ticker.LogLocator(base=2)
    #cbar.update_ticks()
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # Show all ticks and label them with the respective list entries.
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_xticklabels(col_labels)
    ax.set_yticks(np.arange(data.shape[0]))
    ax.set_yticklabels(row_labels)

    # Let the horizontal axes labeling appear on top.
    #ax.tick_params(top=True, bottom=False,
    #               labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=0, ha="center",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    for spine in ax.spines.values():
        spine.set_visible(False)
    
    #label x and y axes
    ax.set_xlabel(xlabel)
    #ax.xaxis.set_label_position('top') 
    ax.set_ylabel(ylabel)
    ax.set_title(title)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=1)
    ax.tick_params(which="minor", bottom=False, left=False)

    #save the figure
    plt.tight_layout()
    plt.savefig(paths['Heatmaps']+'/'+title.replace('/','_').replace('\n','')+'.png',dpi=200)
    
    #optionally - annotate the heatmap w/ text for values
    annotate_heatmap(im,valfmt="{x:.2f}")
    plt.tight_layout()
    plt.savefig(paths['Heatmaps']+'/'+title.replace('/','_').replace('\n','')+'_annotated.png',dpi=200)
    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=("black", "black"),
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A pair of colors.  The first is used for values below a threshold,
        the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = mpl.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

In [5]:
#Needs work - specifically to tell it what TCSs, ligands, etc.
def absorbance_heatmap(dates, paths, tcss, ligand, ligand_concs):
    try:
        abs_data = pd.read_excel('{}/{}_Abs.xlsx'.format(dates[0],dates[0]),sheet_name='Abs',index_col=0)
    except FileNotFoundError:
        print('Absorbance data does not exist. Skipping...')
        return
    
    abs_data = abs_data*3.123
    
    mpl.rcParams.update({'font.size': 8})
    
    ligand_concs.sort()
    fig,ax = plt.subplots()
    im,cbar = heatmap(abs_data.astype(float), tcss, [str(conc) for conc in ligand_concs],
                      '['+ligand+'] (mM)','TCS',
                      tcss[0]+' '+ligand+' OD600 Heatmap',vmax=1.0, vmin=1/100, paths=paths,
                      cbarlabel='OD600',cmap='Blues',origin='lower')
    
    mpl.rcParams.update({'font.size': 12})#Reset font size after plotting abs heatmap

## Transfer Function Functions

In [10]:
def transfer_function(dates,atc,iptg,tcs_b, ligand_b, colors_dict,paths,opt='WTHADN',use_yscale='log',use_ylim=[50,1000]):
    """
    A function to plot transfer functions for data measured in triplicate.
    This will plot all variations of the desired triplicates:
        - Scatterplot w/ error bars
        - Scatterplot w/ error bars and hill fit lines
        - Scatterplot w/ error bars, hill fit lines, autofluorescence line

    Parameters
    ----------
    dates
        The list of dates to pull data from.
    atc
        The concentration of aTc used (ng/mL). Should be the same for all three replicates.
    iptg
        The concentration of IPTG used (uM). Should be the same for all three replicates.
    tcs_b
        A simple string specifying what the base TCS for the curve is. Tells the function what data to look for.
    ligand_b
        A string specifying base ligand to plot for. Used to handle experiments with different ligands on different days.
    colors_dict
        The dictionary of colors to use when plotting data. (May remove in later versions).
    opt
        A string to detemine behavior of the function, based on what data it should plot. Opt can be:
            - 'WTHADN': The function will seek wild-type, HA, and DN data at a series of ligand concentrations to
                plot the transfer function of all three systems vs. [ligand].
            - 'Conc': The function will perform similarly to WTHADN, but will only plot the transfer function vs. [ligand]
                for tcs_b.
            - 'aTc/IPTG': The function will plot the transfer function of tcs_b vs. [aTc] or [IPTG].
    """
    
    data={}
    #Read in all dataframes
    for date in dates:
        data[date] = pd.read_csv('{}/{}_output.csv'.format(date,date),index_col=0)
    
    #Pull out relevant parameters - TCSs, ligand, ligand concentrations
    if opt=='WTHADN':
        tcss = [tcs_b, tcs_b+' HA',tcs_b+' DN']
    else:
        tcss = [tcs_b]
    ligands = {}
    ligand_concs = {}
    print(tcss)
    for tcs in tcss:
        ligands[tcs] = [ligand_b]
        #ligands[tcs] = list(set(data[dates[0]].loc[data[dates[0]]['Plasmid(s)']==tcs]['Ligand'].dropna()))
        #if 'Auto' in ligands[tcs]:
        #    ligands[tcs].remove('Auto')
        #print(data[dates[0]])
        for ligand in ligands[tcs]:
            ligand_concs[ligand] = list(set(data[dates[0]].loc[(data[dates[0]]['Plasmid(s)']==tcs)&(data[dates[0]]['Ligand']==ligand)]['[Ligand] (mM)'].dropna()))
    
    #Get all the data from these dates for these TCSs+Ligands+Concentrations
    #There is almost definitely a better way to structure this, but this way is easy
    to_plot = {}
    for tcs in tcss:
        for ligand in ligands[tcs]:
            for conc in ligand_concs[ligand]:
                to_plot[tcs,ligand,conc] = []
    auto_values = []
    
    for date in dates:
        
        if data[date].loc[(data[date]['Ligand']=='Auto')].empty:
            print(date + ' has no autofluorescence sample. Skipping...')
            #Don't allow this part of the loop to run if one of the replicates lacks autofluorescence sample
            continue
        auto = data[date].loc[data[date]['Ligand']=='Auto']['Gated FL1 Mean'].values[0]
        auto_values.append(auto)
        
        for tcs in tcss:
            for ligand in ligands[tcs]:
                for conc in ligand_concs[ligand]:
                    #print(date+' '+tcs+ligand+str(conc)+' '+str(atc)+str(iptg))
                    #print(data[date].loc[(data[date]['Plasmid(s)']==tcs)&(data[date]['Ligand']==ligand)&
                    #                     (data[date]['[Ligand] (mM)']==conc)&(data[date]['aTc (ng/mL)']==atc)&
                    #                     (data[date]['IPTG (uM)']==iptg)]['Gated FL1 Mean'])
                    gfp=data[date].loc[(data[date]['Plasmid(s)']==tcs)&(data[date]['Ligand']==ligand)&
                                       (data[date]['[Ligand] (mM)']==conc)&(data[date]['aTc (ng/mL)']==atc)&
                                       (data[date]['IPTG (uM)']==iptg)]['Gated FL1 Mean'].values[0]
                    gfp=gfp-auto
                    to_plot[tcs,ligand,conc].append(gfp)

    
    #Begin plotting transfer functions:
    plot_transfer_function(to_plot, ligands[tcs_b], ligand_concs, tcss, colors_dict, paths, opt,use_yscale,use_ylim)

In [11]:
def plot_transfer_function(plot_dict, ligands, ligand_concs, tcss, colors_dict, paths, opt,use_yscale='log',use_ylim=[50,1000]):
    """
    A function to plot transfer functions for data measured in triplicate.
    This can plot all variations of the desired triplicates:
        - Scatterplot w/ error bars
        - Scatterplot w/ error bars and hill fit lines
        - Scatterplot w/ error bars, hill fit lines, autofluorescence line

    Parameters (Needs updating)
    ----------
    plot_dict
        The dictionary containing gfp data associated with ligand concentration. Separated by TCS for ease of use.
        Structure should be: plot_dict[tcs, ligand, ligand_conc] = [gfp1, gfp2, gfp3].
    ligands
        The ligands to be plotted. Will loop over all ligands inside and index into the dict specified in the driver
        function.
    colors_dict
        The dictionary of colors to use when plotting data. (May remove in later versions).
    autofluor
        The values of autofluorescence used to plot the autofluorescence line. False by default meaning
        this line will be omitted from the plot
    hill_fit
        The flag which determines whether the hill function fit should be performed and plotted. False by default
        meaning this curve will be omitted from the plot.
    """
    import lmfit
    
    def hill_residual(params, x, data, eps):
        low = params['Low'].value
        high = params['High'].value
        k = params['K'].value
        n = params['n'].value
        
        prediction = hill_model(x, low, high, k, n)
        #print(prediction)
        return (prediction - data)/eps
    
    #Loop over all ligands
    for ligand in ligands:
        print('Plotting for '+ligand+'...')
        #Preallocate
        data_df={}
        handles = {}
        fig,ax = plt.subplots()
        
        conc_index = ligand_concs[ligand]
        conc_index.sort()
        #print(conc_index)
        curve_df = pd.DataFrame(np.empty([len(ligand_concs[ligand]),len(tcss)]),columns=tcss,index=conc_index)
        curve_df.index.rename('['+ligand+'] (mM)',inplace=True)
        curve_err_df = pd.DataFrame(np.empty([len(ligand_concs[ligand]),len(tcss)]),columns=tcss,index=conc_index)
        curve_err_df.index.rename('['+ligand+'] (mM)',inplace=True)
        params_df = pd.DataFrame(np.empty([len(tcss),5]),columns=['K','n','High','Low','Dynamic Range'],index=tcss)
        params_df.index.rename('TCS',inplace=True)
        inverse = {}
        
        for tcs in tcss:
            print('Plotting '+tcs+'...')
            data_df[tcs]=pd.DataFrame(data=np.empty([10,2]),columns=['['+ligand+'] (mM)','GFP'],index=ligand_concs[ligand])
            for conc in ligand_concs[ligand]:
                data = plot_dict[tcs,ligand,conc]
                gfp = np.mean(data)
                curve_df.at[conc,tcs] = gfp
                std_err = np.std(data)
                curve_err_df.at[conc,tcs] = std_err/np.sqrt(3)
                
                data_df[tcs].at[conc,'['+ligand+'] (mM)'] = conc
                data_df[tcs].at[conc,'GFP'] = gfp
                
                plt.plot(conc,gfp,'o',color=colors_dict[tcs])
                plt.errorbar(conc, gfp, yerr=std_err/np.sqrt(3),ecolor='black',capsize=2)
                
                #Generate handles for a figure legend
                handles[tcs] = mpl.lines.Line2D(xdata=[0, 0],
                                                ydata=[0, 0],
                                                marker='o',
                                                markersize=2.5,
                                                mec=colors_dict[tcs],
                                                mew=0.5,
                                                mfc=mpl.colors.to_rgba('k', alpha=0.),
                                                linewidth=0., color=colors_dict[tcs])
                
            #Determine whether to plot inverse transfer function or normal
            #i.e. check if fluorescence is higher or lower at higher [ligand]
            start = data_df[tcs].at[min(ligand_concs[ligand]),'GFP']
            end = data_df[tcs].at[max(ligand_concs[ligand]),'GFP']
            inverse[tcs] = start > end
            
            #Add some plot parameters like labels, title, scaling        
            plt.xlabel('['+ligand+'] (mM)')
            plt.ylabel('sfGFP Fluorescence (MEFL)')
            #plt.yscale('log')
            #plt.ylim([20, 300])
            plt.yscale(use_yscale)
            plt.ylim(use_ylim)
            plt.title(tcss[0]+' + '+ligand+' Transfer Function')
            plt.tight_layout()
            plt.xscale('symlog',linthresh=0.001)
                
        #This will be the first breakpoint; Add a legend w/o the hill fits
        plt.legend(handles=[handles[tcs] for tcs in tcss],
                   labels=[tcs for tcs in tcss],
                   loc='upper left', bbox_to_anchor=(1,1))
        #need to figure out if I can do plt.savefig but not wipe the plot area.
        
        #Generate hill fits for each TCS
        outs={}
        for tcs in tcss:
            
            if inverse[tcs]:
                print(tcs+' is being fit with an inverse Hill function')
                def hill_model(x, low, high, k, n):
                    return low + (high - low) * (k**n) / (k**n + x**n)
            else:
                print(tcs+' is being fit with a regular Hill function')
                def hill_model(x, low, high, k, n):
                    return low + (high - low) * (x**n) / (k**n + x**n)
            
            print('Fitting Hill function to '+tcs+'...')
            conf = False
            hill_params = lmfit.Parameters()
            hill_params.add('Low', vary=False)
            hill_params.add('High', vary=False)
            hill_params.add('K', value=1, min=0)
            hill_params.add('n', value=1, min=0)
            to_fit = data_df[tcs][['['+ligand+'] (mM)', 'GFP']]
            
            means = pd.pivot_table(data = data_df[tcs],
                                   index = '['+ligand+'] (mM)',
                                   values = 'GFP',
                                   aggfunc = 'mean')
            
            error = to_fit['['+ligand+'] (mM)'].apply(lambda x: means.loc[x]) #means['Gated FL1 Mean'].values
            error = error['GFP'].values
            
            hill_params['Low'].value = means.min().min()
            hill_params['High'].value = means.max().max()
            
            mini = lmfit.Minimizer(hill_residual, 
                                   hill_params,
                                   fcn_args=(to_fit['['+ligand+'] (mM)'].values,
                                             to_fit['GFP'].values,
                                             error))
            
            out = mini.minimize()
            outs[tcs]=out
            out_params = [out.params['Low'],
                          out.params['High'],
                          out.params['K'],
                          out.params['n']] 
            #print(out_params)
            print('The EC50 for '+tcs+' is '+str(out.params['K'].value)+' mM '+ligand)
            print('The dynamic range for '+tcs+' with '+ligand+' is '+str(out.params['High'].value/out.params['Low'].value))
            print('Low: '+str(out.params['Low'].value)+'; High: '+str(out.params['High'].value))
            params_df.at[tcs,'K'] = out.params['K'].value
            params_df.at[tcs,'n'] = out.params['n'].value
            params_df.at[tcs,'High'] = out.params['High'].value
            params_df.at[tcs,'Low'] = out.params['Low'].value
            params_df.at[tcs,'Dynamic Range'] = (out.params['High'].value/out.params['Low'].value)
            #Plot each hill function after generating, onto the plot
            x = np.concatenate((np.linspace(0,0.0001,1000),np.logspace(np.log10(0.0001),np.log10(means.index[-1]),10000)))
            plt.plot(x,hill_model(x,
                                  outs[tcs].params['Low'].value,
                                  outs[tcs].params['High'].value,
                                  outs[tcs].params['K'].value,
                                  outs[tcs].params['n'].value),
                     color=colors_dict[tcs], linewidth=1.5, zorder=-1,label='')
            #Generate handles for a figure legend
            handles[tcs] = mpl.lines.Line2D(xdata=[0, 0],
                                            ydata=[0, 0],
                                            marker='o',
                                            markersize=2.5,
                                            mec=colors_dict[tcs],
                                            mew=0.5,mfc=mpl.colors.to_rgba('k', alpha=0.),
                                            linewidth=1., color=colors_dict[tcs])
        #Plot autofluorescence line
        #plt.axhline(np.mean(auto),color='black',linestyle=':')
        plt.legend(handles=[handles[tcs] for tcs in tcss],
                   labels=[tcs for tcs in tcss],
                   loc='upper left', bbox_to_anchor=(1,1))
        
        if len(data) == 3:
            n = 'Triplicated'
        else:
            n = 'Single-replicate'
        
        if opt == 'WTHADN':
            plt.savefig(paths['Figures']+'/'+tcss[0]+'_HA_DN_'+ligand+'_Fit_'+n+'_TF_wAuto.png',dpi=200,
                        bbox_inches='tight')
            plt.savefig(paths['Figures']+'/'+tcss[0]+'_HA_DN_'+ligand+'_Fit_'+n+'_TF_wAuto.svg',dpi=200,
                        bbox_inches='tight')
        else:
            plt.savefig(paths['Figures']+'/'+tcss[0]+'_'+ligand+'_Fit_'+n+'_TF_wAuto.png',dpi=200,
                        bbox_inches='tight')
            plt.savefig(paths['Figures']+'/'+tcss[0]+'_'+ligand+'_Fit_'+n+'_TF_wAuto.svg',dpi=200, 
                        bbox_inches='tight')
            
        writer = pd.ExcelWriter(paths['IO']+'/'+tcss[0]+'_'+ligand+'_TF.xlsx')
        curve_df.to_excel(writer,sheet_name='Curve Points')
        curve_err_df.to_excel(writer,sheet_name='Standard Error of Mean')
        params_df.to_excel(writer,sheet_name='Parameters')
        #stats_df.to_excel(writer,sheet_name='Curve Stats')
        writer.save()

## Heatmap Function(s)

In [2]:
def make_FL1_heatmaps(paths, minus, plus, folds, fc_output, 
                      full_output=False,tcs=None,ligand=None,conc=None):
    """
    A function to plot FL1 heatmaps.
    This will plot in two different ways depending on the full_output flag:
        - if full_output=False, will look for a tcs, ligand, and conc passed to the function
            and will plot only minus, plus, fold for that combination.
        - Otherwise, will scan through all possible combinations in the minus/plus/folds dicts passed in
            and will plot minus/plus/fold for all of them.

    Parameters
    ----------
    paths
        Paths for directories to save in.
    minus
        Dict of dataframes containing FL1 values for TCS w/o ligand. Structure is minus[tcs,ligand,conc] = dataframe.
    plus
        Dict of dataframes containing FL1 values for TCS w/ ligand. Structure is plus[tcs,ligand,conc] = dataframe.
    folds
        Dict of dataframes containing FL1 fold change values for TCS-ligand. 
        Structure is folds[tcs,ligand,conc] = dataframe.
    minus_hists
    
    plus_hists
    
    full_output (Optional)
        A flag to specify whether to plot heatmaps for only a single (tcs,ligand,conc) or to plot all available data.
        Can be memory-hogging to use full_output=True.
    tcs (optional)
        An optional string to be passed that will specify what TCS to look for data to plot.
    ligand (optional)
        An optional string to be passed that will specify what ligand to look for data to plot.
    conc (optional)
        An optional float to be passed that will specify what conc to look for data to plot.
    """
    
    if not full_output:
        key = (tcs, ligand, conc)
        title_string = tcs+ligand+str(conc)
        fig,ax = plt.subplots()
        heatmap(minus[key].transpose(),minus[key].columns,minus[key].index,'[aTc] (ng/mL)','[IPTG] ($\mu$M)',
                tcs+' (-) '+str(conc)+' mM '+ligand+' mM FL1 Heatmap',10,10000, paths,
                cbarlabel='sfGFP Fluorescence (MEFL)',cmap='Greens')
        
        fig,ax = plt.subplots()
        heatmap(plus[key].transpose(),plus[key].columns,plus[key].index,'[aTc] (ng/mL)','[IPTG] ($\mu$M)',
                tcs+' (+) '+str(conc)+' mM '+ligand+ ' FL1 Heatmap',10,10000, paths,
                cbarlabel='sfGFP Fluorescence (MEFL)',cmap='Greens')
        
        fig,ax = plt.subplots()
        heatmap(folds[key].transpose(),folds[key].columns,folds[key].index,'[aTc] (ng/mL)','[IPTG] ($\mu$M)',
                tcs+' + '+str(conc)+' mM '+ligand+ '\n Fold Change Heatmap',0.1,10, paths,
                cbarlabel='sfGFP Fold Change',cmap='bwr')
        
        #Plot heatmaps consisting of histograms for each TCS+ligand pair
        atc = minus[key].index
        atc = atc.sort_values(ascending=True)
        iptg = minus[key].columns
        iptg = iptg.sort_values(ascending=False)
        
        #Pre-set the indices for subplots that will be on the left hand side, so they can be labeled.
        left_side = [1+i*len(iptg) for i in range(len(iptg))]
        
        fig,ax = plt.subplots()
        k = 1
        for i in iptg:
            for a in atc:
                hist_data = [fc_output.loc[(fc_output['Ligand']==ligand)&(fc_output['[Ligand] (mM)']==0)&
                                           (fc_output['Plasmid(s)']==tcs)&(fc_output['aTc (ng/mL)']==a)&
                                           (fc_output['IPTG (uM)']==i)]['Gated MEFL'].values[0],
                             fc_output.loc[(fc_output['Ligand']==ligand)&(fc_output['[Ligand] (mM)']==conc)&
                                           (fc_output['Plasmid(s)']==tcs)&(fc_output['aTc (ng/mL)']==a)&
                                           (fc_output['IPTG (uM)']==i)]['Gated MEFL'].values[0]]
                
                plt.subplot(len(iptg),len(atc),k)
                
                FlowCal.plot.hist1d(hist_data[0],channel='FL1-H',facecolor=['#00000088'],xlabel='',ylabel='')
                plt.xticks(ticks=[],labels=[])
                plt.yticks(ticks=[],labels=[])
                
                #I only want to label the left-side and bottom row of histograms, to simulate x- and y-ticks & labels
                if k > (len(atc)*len(iptg)-len(atc)):
                    plt.xlabel(str(a))
                if k in left_side:
                    plt.ylabel(str(i),rotation=90)
                k = k+1
        fig.suptitle(tcs+' (-) '+str(conc)+' mM '+ligand+' Histograms')
        fig.supxlabel('[aTc] (ng/mL)')
        fig.supylabel('[IPTG] ($\mu$M)')
        plt.savefig(paths['Heatmaps']+'/'+title_string+'_minh.png',dpi=200)
        plt.savefig(paths['Heatmaps']+'/'+title_string+'_minh.svg',dpi=200)
        plt.show()
        
        fig,ax = plt.subplots()
        k = 1
        for i in iptg:
            for a in atc:
                hist_data = [fc_output.loc[(fc_output['Ligand']==ligand)&(fc_output['[Ligand] (mM)']==0)&
                                           (fc_output['Plasmid(s)']==tcs)&(fc_output['aTc (ng/mL)']==a)&
                                           (fc_output['IPTG (uM)']==i)]['Gated MEFL'].values[0],
                             fc_output.loc[(fc_output['Ligand']==ligand)&(fc_output['[Ligand] (mM)']==conc)&
                                           (fc_output['Plasmid(s)']==tcs)&(fc_output['aTc (ng/mL)']==a)&
                                           (fc_output['IPTG (uM)']==i)]['Gated MEFL'].values[0]]
                
                plt.subplot(len(iptg),len(atc),k)
                FlowCal.plot.hist1d(hist_data[1],channel='FL1-H',facecolor=['#07af6988'],xlabel='',ylabel='')
                plt.xticks(ticks=[],labels=[])
                plt.yticks(ticks=[],labels=[])
                #plt.title(str(k)+' '+str(a)+' '+str(i)) #For debugging
                
                #I only want to label the left-side and bottom row of histograms, to simulate x- and y-ticks & labels
                if k > (len(atc)*len(iptg)-len(atc)):
                    plt.xlabel(str(a))
                if k in left_side:
                    plt.ylabel(str(i),rotation=90)
                k = k+1
        fig.suptitle(tcs+' (+) '+str(conc)+' mM '+ligand+' Histograms')
        fig.supxlabel('[aTc] (ng/mL)')
        fig.supylabel('[IPTG] ($\mu$M)')
        plt.savefig(paths['Heatmaps']+'/'+title_string+'_plush.png',dpi=200)
        plt.savefig(paths['Heatmaps']+'/'+title_string+'_plush.svg',dpi=200)
        plt.show()
        
        fig,ax = plt.subplots()
        k = 1
        for i in iptg:
            for a in atc:
                hist_data = [fc_output.loc[(fc_output['Ligand']==ligand)&(fc_output['[Ligand] (mM)']==0)&
                                           (fc_output['Plasmid(s)']==tcs)&(fc_output['aTc (ng/mL)']==a)&
                                           (fc_output['IPTG (uM)']==i)]['Gated MEFL'].values[0],
                             fc_output.loc[(fc_output['Ligand']==ligand)&(fc_output['[Ligand] (mM)']==conc)&
                                           (fc_output['Plasmid(s)']==tcs)&(fc_output['aTc (ng/mL)']==a)&
                                           (fc_output['IPTG (uM)']==i)]['Gated MEFL'].values[0]]
                
                plt.subplot(len(iptg),len(atc),k)
                FlowCal.plot.hist1d(hist_data,
                                    channel='FL1-H',facecolor=['#00000088','#07af6988'],xlabel='',ylabel='')
                plt.xticks(ticks=[],labels=[])
                plt.yticks(ticks=[],labels=[])
                
                #I only want to label the left-side and bottom row of histograms, to simulate x- and y-ticks & labels
                if k > (len(atc)*len(iptg)-len(atc)):
                    plt.xlabel(str(a))
                if k in left_side:
                    plt.ylabel(str(i),rotation=90)
                k = k+1
        fig.suptitle(tcs+' '+str(conc)+' mM '+ligand+' Histograms')
        fig.supxlabel('[aTc] (ng/mL)')
        fig.supylabel('[IPTG] ($\mu$M)')
        plt.savefig(paths['Heatmaps']+'/'+title_string+'_overlay.png',dpi=200)
        plt.savefig(paths['Heatmaps']+'/'+title_string+'_overlay.svg',dpi=200)
        plt.show()
        
    else:
        for key in list(minus.keys()):
            tcs = key[0]
            ligand = key[1]
            conc = key[2]
            
            fig,ax = plt.subplots()
            heatmap(minus[key].transpose(),minus[key].index,minus[key].columns,'[aTc] (ng/mL)','[IPTG] ($\mu$M)',
                    tcs+' (-) '+ligand+' '+str(conc)+' mM FL1 Heatmap',10,10000, paths,
                    cbarlabel='sfGFP Fluorescence (MEFL)',cmap='Greens')
            
            fig,ax = plt.subplots()
            heatmap(plus[key].transpose(),plus[key].index,plus[key].columns,'[aTc] (ng/mL)','[IPTG] ($\mu$M)',
                    tcs+' (+) '+ligand+' '+str(conc)+' mM FL1 Heatmap',10,10000, paths,
                    cbarlabel='sfGFP Fluorescence (MEFL)',cmap='Greens')
            
            fig,ax = plt.subplots()
            heatmap(folds[key].transpose(),folds[key].index,folds[key].columns,'[aTc] (ng/mL)','[IPTG] ($\mu$M)',
                    tcs+' + '+ligand+' '+str(conc)+' mM FL1 Heatmap',0.1,10, paths,
                    cbarlabel='sfGFP Fold Change',cmap='bwr_r')

### Deprecated Functions
Slated for deletion

In [ ]:
### DEPRECATED - NEEDS TO BE DELETED

def plot_inverse_transfer_function(plot_dict, ligands, ligand_concs, tcss, colors_dict, paths, opt):
    """
    A function to plot transfer functions for data measured in triplicate.
    This can plot all variations of the desired triplicates:
        - Scatterplot w/ error bars
        - Scatterplot w/ error bars and hill fit lines
        - Scatterplot w/ error bars, hill fit lines, autofluorescence line

    Parameters (Needs updating)
    ----------
    plot_dict
        The dictionary containing gfp data associated with ligand concentration. Separated by TCS for ease of use.
        Structure should be: plot_dict[tcs, ligand, ligand_conc] = [gfp1, gfp2, gfp3].
    ligands
        The ligands to be plotted. Will loop over all ligands inside and index into the dict specified in the driver
        function.
    colors_dict
        The dictionary of colors to use when plotting data. (May remove in later versions).
    autofluor
        The values of autofluorescence used to plot the autofluorescence line. False by default meaning
        this line will be omitted from the plot
    hill_fit
        The flag which determines whether the hill function fit should be performed and plotted. False by default
        meaning this curve will be omitted from the plot.
    """
    import lmfit
    
    def hill_model(x, low, high, k, n):
        #print(str(x)+' '+str(low)+' '+str(high)+' '+str(k)+' '+str(n))
        return low + (high - low) * (k**n) / (k**n + x**n)

    def hill_residual(params, x, data, eps):
        low = params['Low'].value
        high = params['High'].value
        k = params['K'].value
        n = params['n'].value
    
        prediction = hill_model(x, low, high, k, n)
        #print(prediction)
        return (prediction - data)/eps
    
    #Loop over all ligands
    for ligand in ligands:
        print('Plotting for '+ligand+'...')
        #Preallocate
        data_df={}
        handles = {}
        fig,ax = plt.subplots()
        
        curve_df = pd.DataFrame(np.empty([len(ligand_concs[ligand]),len(tcss)]),columns=tcss,index=ligand_concs[ligand])
        curve_df.index.rename('['+ligand+'] (mM)',inplace=True)
        curve_err_df = pd.DataFrame(np.empty([len(ligand_concs[ligand]),len(tcss)]),columns=tcss,index=ligand_concs[ligand])
        curve_err_df.index.rename('['+ligand+'] (mM)',inplace=True)
        
        for tcs in tcss:
            print('Plotting '+tcs+'...')
            data_df[tcs]=pd.DataFrame(data=np.empty([10,2]),columns=['['+ligand+'] (mM)','GFP'],index=ligand_concs[ligand])
            for conc in ligand_concs[ligand]:
                data = plot_dict[tcs,ligand,conc]
                gfp = np.mean(data)
                curve_df.at[conc,tcs] = gfp
                std_err = np.std(data)
                curve_err_df.at[conc,tcs] = std_err/np.sqrt(3)
                
                data_df[tcs].at[conc,'['+ligand+'] (mM)'] = conc
                data_df[tcs].at[conc,'GFP'] = gfp
                
                plt.plot(conc,gfp,'o',color=colors_dict[tcs])
                plt.errorbar(conc, gfp, yerr=std_err/np.sqrt(3),ecolor='black',capsize=2)
                
                #Generate handles for a figure legend
                handles[tcs] = mpl.lines.Line2D(xdata=[0, 0],
                                                ydata=[0, 0],
                                                marker='o',
                                                markersize=2.5,
                                                mec=colors_dict[tcs],
                                                mew=0.5,
                                                mfc=mpl.colors.to_rgba('k', alpha=0.),
                                                linewidth=0., color=colors_dict[tcs])
                
                #Add some plot parameters like labels, title, scaling        
                plt.xlabel('['+ligand+'] (mM)')
                plt.ylabel('sfGFP Fluorescence (MEFL)')
                plt.yscale('log')
                plt.title(tcss[0]+' + '+ligand+' Transfer Function')
                plt.tight_layout()
                plt.xscale('symlog',linthresh=0.01)
                
        #This will be the first breakpoint; Add a legend w/o the hill fits
        plt.legend(handles=[handles[tcs] for tcs in tcss],
                   labels=[tcs for tcs in tcss],
                   loc='upper left', bbox_to_anchor=(1,1))
        #need to figure out if I can do plt.savefig but not wipe the plot area.
        
        #Generate hill fits for each TCS
        outs={}
        for tcs in tcss:
            print('Fitting Hill function to '+tcs+'...')
            conf = False
            hill_params = lmfit.Parameters()
            hill_params.add('Low', vary=False)
            hill_params.add('High', vary=False)
            hill_params.add('K', value=1, min=0)
            hill_params.add('n', value=1, min=0)
            to_fit = data_df[tcs][['['+ligand+'] (mM)', 'GFP']]
            
            means = pd.pivot_table(data = data_df[tcs],
                                   index = '['+ligand+'] (mM)',
                                   values = 'GFP',
                                   aggfunc = 'mean')
            
            error = to_fit['['+ligand+'] (mM)'].apply(lambda x: means.loc[x]) #means['Gated FL1 Mean'].values
            error = error['GFP'].values
            
            hill_params['Low'].value = means.min().min()
            hill_params['High'].value = means.max().max()
            
            mini = lmfit.Minimizer(hill_residual, 
                                   hill_params,
                                   fcn_args=(to_fit['['+ligand+'] (mM)'].values,
                                             to_fit['GFP'].values,
                                             error))
            
            out = mini.minimize()
            outs[tcs]=out
            out_params = [out.params['Low'],
                          out.params['High'],
                          out.params['K'],
                          out.params['n']] 
            #Plot each hill function after generating, onto the plot
            x = np.concatenate((np.linspace(0,1,10000),np.linspace(1,means.index[-1],10)))
            plt.plot(x,hill_model(x,
                                  outs[tcs].params['Low'].value,
                                  outs[tcs].params['High'].value,
                                  outs[tcs].params['K'].value,
                                  outs[tcs].params['n'].value),
                     color=colors_dict[tcs], linewidth=1.5, zorder=-1,label='')
            #Generate handles for a figure legend
            handles[tcs] = mpl.lines.Line2D(xdata=[0, 0],
                                            ydata=[0, 0],
                                            marker='o',
                                            markersize=2.5,
                                            mec=colors_dict[tcs],
                                            mew=0.5,mfc=mpl.colors.to_rgba('k', alpha=0.),
                                            linewidth=1., color=colors_dict[tcs])
        #Plot autofluorescence line
        #plt.axhline(np.mean(auto),color='black',linestyle=':')
        plt.legend(handles=[handles[tcs] for tcs in tcss],
                   labels=[tcs for tcs in tcss],
                   loc='upper left', bbox_to_anchor=(1,1))
        
        if len(data) == 3:
            n = 'Triplicated'
        else:
            n = 'Single-replicate'
        
        if opt == 'WTHADN':
            plt.savefig(paths['Figures']+'/'+tcss[0]+'_HA_DN_'+ligand+'_Fit_'+n+'_TF_wAuto.png',dpi=200,
                        bbox_inches='tight')
            plt.savefig(paths['Figures']+'/'+tcss[0]+'_HA_DN_'+ligand+'_Fit_'+n+'_TF_wAuto.svg',dpi=200,
                        bbox_inches='tight')
        else:
            plt.savefig(paths['Figures']+'/'+tcss[0]+'_'+ligand+'_Fit_'+n+'_TF_wAuto.png',dpi=200,
                        bbox_inches='tight')
            plt.savefig(paths['Figures']+'/'+tcss[0]+'_'+ligand+'_Fit_'+n+'_TF_wAuto.svg',dpi=200, 
                        bbox_inches='tight')
        plt.show()
        
        writer = pd.ExcelWriter(paths['IO']+'/'+tcss[0]+'_'+ligand+'_TF.xlsx')
        curve_df.to_excel(writer,sheet_name='Curve Points')
        curve_err_df.to_excel(writer,sheet_name='Standard Error of Mean')
        #stats_df.to_excel(writer,sheet_name='Curve Stats')
        writer.save()